In [1]:
## Simulate data for different genotype groups

# Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"



Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel

corrplot 0.95 loaded

Loading required package: viridisLite


Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose



Attaching package: ‘MASS’


The following object is masked from ‘package:genefilter’:

    area


The following object is masked from ‘package:biomaRt’:

    select



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following object is masked from ‘package:biomaRt’:

    selec

# Parameters

In [3]:
### Path to summary statistics of single-cell datasets

data_path_sc = "../data/current/summary_stats/"

In [5]:
### Path to save simulation results
output_path_sc = '../results/current/A1/'


# Functions

In [6]:
source('MS1_Functions.r')

# Data

## Get summary statistics of input data

In [ ]:
### Define parameters based on which cell-type and dataset to run simulations

In [7]:
dataset = 'wijst2018'
cell_type = 'B'

### Amount cells per sample and cell-type

In [ ]:
### For selected dataset and cell-type get the amount of cells per sample summary

In [10]:
sample_summary = read.csv(paste0(data_path_sc, 'D1_Sample_summarywg3_', dataset, cell_type, '.Qced.Normalized.SCs.Rds.csv'))

In [11]:
sample_summary$X = NULL

In [12]:
head(sample_summary,2)

,Assignment,amount_cells,dataset,cell_type
,<chr>,<int>,<chr>,<chr>
1,LL001060,14,wg3_wijst2018,B.Qced.Normalized.SCs.Rds
2,LL002176,14,wg3_wijst2018,B.Qced.Normalized.SCs.Rds


In [13]:
sample_summary$cell_type = str_replace(sample_summary$cell_type, '\\..*', '')

### Statistics per sample and gene

In [14]:
## For selected dataset and cell-type get for each sample and gene summary information, like mean expression, percentage zero, ...

In [15]:
assay = ''

In [16]:
if(assay != ''){
    sample_gene_statistic = read.csv(paste0(data_path_sc, 'D1_Sample_gene_statisticwg3_',dataset, cell_type, '.Qced.Normalized.SCs.Rds_', assay, '.csv'))
    }
if(assay == ''){
    sample_gene_statistic = read.csv(paste0(data_path_sc, 'D1_Sample_gene_statisticwg3_',dataset, cell_type, '.Qced.Normalized.SCs.Rds', '.csv'))
    }
    

In [17]:
head(sample_gene_statistic,2)

,X,gene,amount_cells,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,amount_cells_bin,percentage_zero_bin,sample,cell_type,dataset
,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,RPL22,13,0,6.461538,17,2,16.26923,0,13,"(0,25]","(-0.001,5]",LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,2,RPL11,13,0,26.461538,47,11,105.76923,0,13,"(0,25]","(-0.001,5]",LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018


In [18]:
### Calculate the  amount genes per sample, cell_type, dataset (after filtering)

In [19]:
amount_genes_summary = sample_gene_statistic %>% group_by(sample, cell_type, dataset) %>% summarise(amount_genes = n())

`summarise()` has grouped output by 'sample', 'cell_type'. You can override
using the `.groups` argument.


In [20]:
amount_genes_summary$cell_type = str_replace(amount_genes_summary$cell_type, '\\..*', '')

In [21]:
head(amount_genes_summary,2)

sample,cell_type,dataset,amount_genes
<chr>,<chr>,<chr>,<int>
LL001060,B,wg3_wijst2018,238
LL002176,B,wg3_wijst2018,227


In [22]:
### Generate new columns

In [23]:
sample_gene_statistic$percentage_zero = sample_gene_statistic$percentage_zero*100

In [24]:
## Binarize the amount of cells and percentage of zero columns
sample_gene_statistic = sample_gene_statistic %>% mutate(amount_cells_bin = cut(amount_cells, breaks=c(0,25, 50, 100,250, 500, 750, 1500)))
sample_gene_statistic = sample_gene_statistic %>% mutate(percentage_zero_bin = cut(percentage_zero, breaks=c(-0.001,5, 10, 20,40, 60, 80,90, 95, 100)))

In [25]:
unique(sample_gene_statistic$amount_cells_bin)

[1] (0,25]  (25,50]
7 Levels: (0,25] (25,50] (50,100] (100,250] (250,500] ... (750,1.5e+03]

In [26]:
head(sample_gene_statistic,2)

,X,gene,amount_cells,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,amount_cells_bin,percentage_zero_bin,sample,cell_type,dataset
,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<fct>,<fct>,<chr>,<chr>,<chr>
1,1,RPL22,13,0,6.461538,17,2,16.26923,0,13,"(0,25]","(-0.001,5]",LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,2,RPL11,13,0,26.461538,47,11,105.76923,0,13,"(0,25]","(-0.001,5]",LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018


In [27]:
sample_gene_statistic$cell_type_short = str_replace(sample_gene_statistic$cell_type, '\\..*', '')

### Statistics per cell and sample

In [ ]:
## For selected dataset and cell-type get for each cell information, like mean counts, max counts, amount of non-zero genes

In [28]:
if(assay != ''){
    sample_cell_statistic = read.csv(paste0(data_path_sc, 'D1_Sample_cell_statisticwg3_', dataset, cell_type, '.Qced.Normalized.SCs.Rds_', assay, '.csv'))
    }
if(assay == ''){
    sample_cell_statistic = read.csv(paste0(data_path_sc, 'D1_Sample_cell_statisticwg3_', dataset, cell_type, '.Qced.Normalized.SCs.Rds' ,  '.csv'))
    }
    

In [29]:
sample_cell_statistic$cell_type_short = str_replace(sample_cell_statistic$cell_type, '\\..*', '')

In [30]:
mean_count_per_cell = sample_cell_statistic %>% group_by(dataset, cell_type, cell_type_short) %>% summarise(total_mean_count = mean(mean_counts))

`summarise()` has grouped output by 'dataset', 'cell_type'. You can override
using the `.groups` argument.


In [31]:
head(mean_count_per_cell,2)

dataset,cell_type,cell_type_short,total_mean_count
<chr>,<chr>,<chr>,<dbl>
wg3_wijst2018,B.Qced.Normalized.SCs.Rds,B,0.2825653


In [32]:
sample_cell_statistic = merge(sample_cell_statistic, mean_count_per_cell)

In [33]:
head(sample_cell_statistic,2)

,cell_type,dataset,cell_type_short,X,cell,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,amount_non_zero_counts,sample,total_mean_count
,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<dbl>
1,B.Qced.Normalized.SCs.Rds,wg3_wijst2018,B,1,AAAGCAAGTCGCATCG_pilot3_lane1,5216,14699,13237,0.3548541,322,0,16.401625,0.9005375,1462,LL007985,0.2825653
2,B.Qced.Normalized.SCs.Rds,wg3_wijst2018,B,2,AACTTTCTCACAATGC_pilot3_lane1,3848,14699,13536,0.2617865,217,0,8.229191,0.9208790,1163,LL007985,0.2825653


In [ ]:
### Calculate scaling factor: the mean counts of a cell compared to the total mean counts across all cells

In [34]:
sample_cell_statistic$scaling_factor = sample_cell_statistic$total_mean_count / sample_cell_statistic$mean_counts

In [35]:
head(sample_cell_statistic$scaling_factor)

[1] 0.7962858 1.0793729 1.2435410 1.0718521 0.6028196 1.0644354

## Estimation of mu and size for neg binom parameters

In [36]:
## For selected dataset and cell-type get for each sample and gene summary information, like mean expression, percentage zero, ... (including estimated mu and size parameter with negative binomial distribution)

In [37]:
assay = 'RNA'

In [38]:
if(assay != ''){
    sample_gene_statistic_binom = read.csv(paste0(data_path_sc, 'D1_Sample_gene_statistic_binomwg3_',dataset, cell_type, '.Qced.Normalized.SCs.Rds_', assay, '_random.csv'))
    }
if(assay == ''){
    sample_gene_statistic_binom = read.csv(paste0(data_path_sc, 'D1_Sample_gene_statistic_binomwg3_',dataset, cell_type, '.Qced.Normalized.SCs.Rds', '_random.csv'))
    }
    

In [39]:
head(sample_gene_statistic_binom,2)

,X,gene,amount_cells,amount_zero_counts,mean_counts,max_counts,min_counts,variance_counts,bin_size,bin_mu,percentage_zero,amount_non_zero_counts,amount_cells_bin,percentage_zero_bin,sample,cell_type,dataset
,<int>,<chr>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,FTL,13,0,18.15385,66,2,398.97436,1.068951,18.15385,0,13,"(0,25]","(-0.001,5]",LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018
2,2,MT-ND2,13,0,10.23077,25,4,36.52564,5.087908,10.23077,0,13,"(0,25]","(-0.001,5]",LL007985,B.Qced.Normalized.SCs.Rds,wg3_wijst2018


# Simulate Data with co-eQTL effect

## Simulation (copula approach)

### Generate simulated data 

#### Parameters

In [40]:
### Additions to file-names; no other implications

In [41]:
use_approach = 'wijstB4'    # addition to file-name

In [42]:
sample_data = TRUE

In [43]:
### Define parameters with effect on simulation

In [44]:
## Amount of individuals per genotype

In [45]:
ind_genotype_spec  = data.frame(n_ind = c(20,15,10),
                                genotypes = c('XX','XY', 'YY')) ## MAX: 3 different genotypes

In [46]:
## Amount of gene pairs and genes with diff correlation between genotypes and diff expression

In [47]:
gene_pairs = 10000

In [ ]:
### For each genotype specify the correlation between gene-pairs

In [48]:
cor_spec = data.frame(genotypes = c('XX', 'XY', 'XY',   'YY', 'YY'),
           gene_pairs = c(100, 50, 50, 50, 50),   # percentage of gene-pairs with this correlation; ! in case of 3 genotypes not two different percentage splits
           correlations = c(0,  0,0.05,0,  0.1))

In [49]:
cor_spec

genotypes,gene_pairs,correlations
<chr>,<dbl>,<dbl>
XX,100,0.00
XY,50,0.00
XY,50,0.05
YY,50,0.00
YY,50,0.10


In [50]:
eqtl_effect = 100 ## Defines percentage of gene-pairs with eQTL effect on gene1; change in expression between genotypes

In [51]:
### Amount of cells across individuals (currently no differences between genotypes possible)

In [52]:
vary_ncells = TRUE  # have individuals with different amount of cells

In [53]:
ncells_sd = 9 # standard deviation in amount of cells of individuals

In [54]:
ncells_range = c(16) # mean amount of cells of indiviuals

In [55]:
### Paremeters for simulation (keep fixed)

In [56]:
var_norm_range = c( 1) # sd of normal distribution keep this at 1

if(sample_data == FALSE){
    size_par = 12   # size parameter for binomial distribution
    }


if(sample_data == TRUE){
    size_par = sample_gene_statistic_binom$bin_size[!is.na(sample_gene_statistic_binom$bin_size)]   # size parameter for binomial distribution
    }

In [57]:
#### Range of mean values and combine with size for the different genes

In [58]:
## set a range of base mean value (either fixed values / or sample from real data estimates)
#basemean_range1 <- c( 12, 4, 3,  2,1.75, 1.5, 1, 0.75,0.5,0.25,0.15, 0.1,0.075,  0.05, 0.01)
#basemean_range2<- c( 12, 4, 3, 2,1.75, 1.5, 1, 0.75,0.5,0.25,0.15, 0.1,0.075, 0.05, 0.01)

if(sample_data == TRUE){
    basemean_range1 <- sample_gene_statistic_binom$bin_mu[!is.na(sample_gene_statistic_binom$bin_mu)]
    basemean_range2<- sample_gene_statistic_binom$bin_mu[!is.na(sample_gene_statistic_binom$bin_mu)]
    }

if(sample_data == FALSE){
    basemean_range1 <- c(24,16, 12,9, 6, 4,3, 2, 1, 0.5,0.25, 0.1, 0.05, 0.01)
    basemean_range2<- c(24 ,16, 12,9, 6, 4,3, 2, 1, 0.5,0.25, 0.1, 0.05, 0.01)
    }

basemean_range = c(basemean_range1, basemean_range2)

basemean_size_data = data.frame(mu = basemean_range, size = size_par)

In [59]:
exp(2.7)

[1] 14.87973

In [60]:
head(basemean_size_data,2)

,mu,size
,<dbl>,<dbl>
1,18.15385,1.068951
2,10.23077,5.087908


In [61]:
nrow(basemean_size_data)

[1] 10776

In [62]:
### Seed for reproducibility

In [63]:
seed_var = 2 # Change this for each genotype/ simulation run; ensures that individuals are different (effect on randomly defining amount of cells per individual)

In [64]:
### Affecting simulation

In [65]:
### Define whether each cell gets a different number of counts
variant = 'scaling' #OPTIONS: limited_scaling, no_scaling, scaling ; defines differences in amount of counts between cells: scaling = amount of counts are multiplied with scaling factor from data

In [66]:
use_data_estimates = FALSE   # Define basemean based on means in data vs. pre-specified range of base-means

#### Specify gene-pair correlations and means / size per individual

In [67]:
### Generate a data.frame of the gene-pairs

In [68]:
specification = data.frame(gene1 = paste0('gene1_', 1:gene_pairs), gene2 = paste0(gene2 =  paste0('gene2_', 1:gene_pairs)))

In [69]:
head(specification,2)

,gene1,gene2
,<chr>,<chr>
1,gene1_1,gene2_1
2,gene1_2,gene2_2


In [70]:
nrow(specification)

[1] 10000

In [71]:
### Add initial configuration of mean and size

In [72]:
set.seed(1)
specification$gene1_mu = sample(basemean_range1, nrow(specification), replace = TRUE)
specification$gene2_mu = sample(basemean_range2, nrow(specification), replace = TRUE)

In [73]:
nrow(specification[specification$gene1_mu == specification$gene2_mu,])  # amount of occurrences with same mean

[1] 1

In [74]:
### Add size (sample either randomly from data or choose fixed parameter)
set.seed(1)
specification$gene1_size = sample(size_par, nrow(specification), replace = TRUE)
specification$gene2_size = sample(size_par, nrow(specification), replace = TRUE)

In [75]:
summary(size_par)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
   0.0928    2.5545    7.0663   35.6014  100.0000 1948.4706 

In [76]:
head(specification)

,gene1,gene2,gene1_mu,gene2_mu,gene1_size,gene2_size
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,gene1_1,gene2_1,27.1666667,1.9444295,12.4576862,3.781396
2,gene1_2,gene2_2,0.4074096,0.6500158,100.0000050,100.000223
3,gene1_3,gene2_3,2.0769239,0.5332714,0.8328116,5.503608
4,gene1_4,gene2_4,2.1852051,1.1111171,1.7217598,3.994068
5,gene1_5,gene2_5,44.8333333,8.4722223,6.8704182,4.469030
6,gene1_6,gene2_6,0.5172510,14.5294118,17.1139476,1.869628


In [78]:
### Modify the base-specification for individuals of different genotypes

In [79]:
specification_data_all = data.frame()

In [80]:
cor_spec

genotypes,gene_pairs,correlations
<chr>,<dbl>,<dbl>
XX,100,0.00
XY,50,0.00
XY,50,0.05
YY,50,0.00
YY,50,0.10


In [81]:
for(i in unique(cor_spec$genotypes)){
    specification_genotype = specification
    specification_genotype$genotype = i
    
    ### Set up the correlations of the gene-pairs based on the specified proportions
    current_genotype = i
    num_rows = cor_spec$gene_pairs[cor_spec$genotypes == i]
    correlation_value = cor_spec$correlations[cor_spec$genotypes == i]
    
    amount_correlations = nrow(specification_genotype)
    
    # Calculate proportions
    proportions = num_rows / sum(num_rows)

    # Generate the output vector of correlations
    set.seed(1)
    output_vector = sample(
      correlation_value,
      size =  sum(num_rows),
      replace = TRUE,
      prob = proportions
    )
    
    # Add the output vector of correlations to the specification
    specification_genotype$correlation = output_vector
    
    specification_data_all = rbind(specification_genotype, specification_data_all)
    
    
    }   

In [82]:
 sum(num_rows)

[1] 100

In [83]:
nrow(specification_data_all)

[1] 30000

In [84]:
test = specification_data_all %>% group_by(gene1, gene2,  correlation) %>% dplyr::count()

In [85]:
unique(test$n)

[1] 1 3

In [86]:
specification_data_all %>% group_by(correlation, genotype) %>% dplyr::count()

correlation,genotype,n
<dbl>,<chr>,<int>
0.00,XX,10000
0.00,XY,4800
0.00,YY,4800
0.05,XY,5200
0.10,YY,5200


In [87]:
nrow(test[test$n == 3,])

[1] 4800

In [88]:
nrow(test[test$n == 2,])

[1] 0

In [89]:
nrow(test[test$n == 1,])

[1] 15600

In [90]:
specification_data_all$gene_pair = paste0(specification_data_all$gene1, ';', specification_data_all$gene2)

In [91]:
head(specification_data_all,6)

,gene1,gene2,gene1_mu,gene2_mu,gene1_size,gene2_size,genotype,correlation,gene_pair
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,gene1_1,gene2_1,27.1666667,1.9444295,12.4576862,3.781396,YY,0.1,gene1_1;gene2_1
2,gene1_2,gene2_2,0.4074096,0.6500158,100.0000050,100.000223,YY,0.1,gene1_2;gene2_2
3,gene1_3,gene2_3,2.0769239,0.5332714,0.8328116,5.503608,YY,0.0,gene1_3;gene2_3
4,gene1_4,gene2_4,2.1852051,1.1111171,1.7217598,3.994068,YY,0.0,gene1_4;gene2_4
5,gene1_5,gene2_5,44.8333333,8.4722223,6.8704182,4.469030,YY,0.1,gene1_5;gene2_5
6,gene1_6,gene2_6,0.5172510,14.5294118,17.1139476,1.869628,YY,0.0,gene1_6;gene2_6


In [92]:
specification_data_all %>% group_by(genotype, correlation) %>% dplyr::count()

genotype,correlation,n
<chr>,<dbl>,<int>
XX,0.00,10000
XY,0.00,4800
XY,0.05,5200
YY,0.00,4800
YY,0.10,5200


In [93]:
### Add eQTL effect if specified

In [94]:
eqtl_effect

[1] 100

In [95]:
### Select percentage of gene-pairs with eQTL effect

In [96]:
set.seed(1)
sel_gene_pairs = sample(specification_data_all$gene_pair, (eqtl_effect/100) * length(unique(specification_data_all$gene_pair)))

In [97]:
length(sel_gene_pairs)

[1] 10000

In [98]:
### Add different gene1_MU values for those selected gene-pairs across the genotypes

In [99]:
set.seed(1)
for(i in unique(sel_gene_pairs)){
    
    modification_data =  specification_data_all[specification_data_all$gene_pair %in% i, ] # select the gene-pair that should be modified
    specification_data_all = specification_data_all[!specification_data_all$gene_pair %in% i, ] # remove the gene-pair that will be modified
    
    ## Order modification_data
    modification_data = modification_data  %>% arrange(gene1, gene2, correlation, genotype)
    
    ## get a new range of basemean values for modification data
    mu_values = sample(basemean_range1, nrow(modification_data)) ## removed sorting which could lead to one genotype always getting higher base-mean
    
    # Randomly decide ascending or descending order (to make sure sometimes higher/ lower expression for genotype)
    if (sample(c(TRUE, FALSE), 1)) {
      # Sort in ascending order
      mu_values <- mu_values[order(mu_values)]
    } else {
      # Sort in descending order
      mu_values <- mu_values[order(-mu_values)]
    }
    
    ## put new basemean value
    modification_data$gene1_mu = mu_values
    
    ### Add modification data to the specification data
    specification_data_all = rbind(specification_data_all, modification_data)
    print(i)
    flush.console()
    }
    
    
    

[1] "gene1_7401;gene2_7401"
[1] "gene1_4388;gene2_4388"
[1] "gene1_4775;gene2_4775"
[1] "gene1_6753;gene2_6753"
[1] "gene1_3218;gene2_3218"
[1] "gene1_6109;gene2_6109"
[1] "gene1_9143;gene2_9143"
[1] "gene1_539;gene2_539"
[1] "gene1_8462;gene2_8462"
[1] "gene1_4050;gene2_4050"
[1] "gene1_3499;gene2_3499"
[1] "gene1_1571;gene2_1571"
[1] "gene1_2257;gene2_2257"
[1] "gene1_5173;gene2_5173"
[1] "gene1_7685;gene2_7685"
[1] "gene1_4262;gene2_4262"
[1] "gene1_4906;gene2_4906"
[1] "gene1_8183;gene2_8183"
[1] "gene1_8489;gene2_8489"
[1] "gene1_3903;gene2_3903"
[1] "gene1_9941;gene2_9941"
[1] "gene1_8229;gene2_8229"
[1] "gene1_7513;gene2_7513"
[1] "gene1_5305;gene2_5305"
[1] "gene1_878;gene2_878"
[1] "gene1_5061;gene2_5061"
[1] "gene1_4229;gene2_4229"
[1] "gene1_2306;gene2_2306"
[1] "gene1_6526;gene2_6526"
[1] "gene1_2204;gene2_2204"
[1] "gene1_1455;gene2_1455"
[1] "gene1_1041;gene2_1041"
[1] "gene1_9284;gene2_9284"
[1] "gene1_7075;gene2_7075"
[1] "gene1_4662;gene2_4662"
[1] "gene1_6954;gene2_69

In [100]:
nrow(specification_data_all)

[1] 30000

In [101]:
test = specification_data_all %>% group_by(gene_pair, gene1_mu) %>% dplyr::count()

In [102]:
nrow(test[test$n == 1,])

[1] 21236

In [103]:
nrow(test[test$n == 3,])

[1] 2918

In [104]:
## Get a specificatoin for each individual

In [105]:
head(specification_data_all,2)

,gene1,gene2,gene1_mu,gene2_mu,gene1_size,gene2_size,genotype,correlation,gene_pair
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,gene1_1,gene2_1,27.166667,1.944429,12.45769,3.781396,YY,0.1,gene1_1;gene2_1
4,gene1_4,gene2_4,2.185205,1.111117,1.72176,3.994068,YY,0.0,gene1_4;gene2_4


In [106]:
final_specification = data.frame()

In [107]:
for(i in ind_genotype_spec$genotypes){
    for(j in 1:ind_genotype_spec$n_ind[ind_genotype_spec$genotypes == i]){
        data = specification_data_all[specification_data_all$genotype == i,]
        data$ind = j
        final_specification = rbind(final_specification, data)
        }
    }
        

In [108]:
nrow(final_specification)

[1] 450000

In [109]:
head(final_specification,2)

,gene1,gene2,gene1_mu,gene2_mu,gene1_size,gene2_size,genotype,correlation,gene_pair,ind
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<int>
20001,gene1_1,gene2_1,27.166667,1.944429,12.45769,3.781396,XX,0,gene1_1;gene2_1,1
20004,gene1_4,gene2_4,2.185205,1.111117,1.72176,3.994068,XX,0,gene1_4;gene2_4,1


In [110]:
ind_genotype_spec 

n_ind,genotypes
<dbl>,<chr>
20,XX
15,XY
10,YY


#### Sample scaling factors from data for simulation

In [111]:
## Get example scaling parameters from data
scaling_par = sample_cell_statistic$scaling_factor

In [112]:
head(scaling_par,2)

[1] 0.7962858 1.0793729

In [113]:
## Limit range of scaling parameter for certain range
if(variant == 'limited_scaling'){
    scaling_par = scaling_par[(scaling_par > 0.5) & (scaling_par < 2)]
    }

if(variant == 'no_scaling'){
    scaling_par = rep(1, length(scaling_par))
    }

In [114]:
#correlations

#### Simulate data

In [115]:
## Empty datasets to store information

count_data = data.frame()
gene_summary_all_cells = data.frame()
gene_summary_all = data.frame()
cell_summary_all_cells = data.frame()
cell_summary_all = data.frame()

In [116]:
count_data

<0 x 0 matrix>

In [117]:
use_approach

[1] "wijstB4"

In [118]:
head(final_specification,2)

,gene1,gene2,gene1_mu,gene2_mu,gene1_size,gene2_size,genotype,correlation,gene_pair,ind
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<int>
20001,gene1_1,gene2_1,27.166667,1.944429,12.45769,3.781396,XX,0,gene1_1;gene2_1,1
20004,gene1_4,gene2_4,2.185205,1.111117,1.72176,3.994068,XX,0,gene1_4;gene2_4,1


In [119]:
final_specification$ident = paste0(final_specification$genotype, ';', final_specification$ind)

In [120]:
### save the final specification
write.csv(final_specification, paste0(output_path_sc, '/A1_4_Simulation_Specified_Correlation', use_approach,'_',  variant, '.csv'))


In [121]:
#head(mu_long,2)

In [122]:
set.seed(seed_var)
for(j in unique(final_specification$ident)){  # put this for loop before correlation matrix specification if there should be variation of correlations for diff inds. in true data    
    ### Define amount of cells for the individual and the scaling for each cell    
    print(j)
    flush.console()
    ncells = ncells_range
        
    if(vary_ncells == TRUE){
        ncells = max(round(rnorm(1, mean = ncells_range, sd = ncells_sd)),10)
        }

    scaling_data = data.frame(scaling = sample(scaling_par, ncells), cell_ident = paste0('cell_',j, '_', ncells, '_',  1:ncells))  # select subset of scaling parameter depending on ncells

    ### Get the data for the cor_matrices
    data_ind = final_specification[final_specification$ident == j,]
    data_ind_list = split(data_ind, 1:nrow(data_ind))   # split to have a list element for each row
    
    ### Specify a correlatin matrix for each row
    ## cor matrix for each row
    cor_matrices = lapply(data_ind_list , function(x){
        cor_matrix = matrix( c(1, x$correlation, x$correlation, 1), nrow = 2)
        rownames(cor_matrix) = c(x$gene1, x$gene2)
        colnames(cor_matrix) = c(x$gene1, x$gene2)
        return(cor_matrix)
        })


    ### Get normally distributed correlated values for each cor_matrix/ gene-pair
    norm_data = lapply(cor_matrices, function(x){
        mu =  rnorm_multi(n = ncells, 
                  mu = rep(0,ncol(x)),
                  sd = rep(1, ncol(x)),
                  r = unlist(c(x)), 
                  varnames = rownames(x),
                  empirical = TRUE)
        })


    ### Convert to probabilities and long format
    prob_data = lapply(norm_data, function(x){
        mu = pnorm(as.matrix(x), mean = 0, sd = 1)
        mu = as.data.frame(mu)
        mu$cell_ident = paste0('cell_',j, '_', ncells, '_', 1:nrow(mu))
        mu = melt(mu, id = 'cell_ident')
        return(mu)
        })

    ### Add scaling and get counts
    mu_long = as.data.frame(data.table::rbindlist(prob_data))

       
    mu_long = merge(mu_long, scaling_data)
    
    ### Add the mean and size
    mean_size_specification = rbind( data.frame(variable = data_ind$gene1, mu = data_ind$gene1_mu, size = data_ind$gene1_size),
                                data.frame(variable = data_ind$gene2, mu = data_ind$gene2_mu, size = data_ind$gene2_size))
    
    nrow(mu_long)
    mu_long = merge(mu_long, mean_size_specification)
    nrow(mu_long)
    

    mu_long = mu_long %>% group_by(variable) %>% mutate(
                counts = qnbinom(value, size =as.numeric(size), mu = as.numeric(mu) / scaling
                                ),
                basemean = as.numeric(mu),
                size = as.numeric(size)
                )
    mu_long$ncells = ncells
    mu_long$ind = j
    #mu_long$genotype = genotype
    
    ### Save data per individual and ncells
    #write.csv(mu_long, paste0(output_path_sc, '/A1_4_Simulated_Counts', use_approach, j, ncells,  variant,  '.csv'))

        
    ### Generate and save gene and cell summaries for simulated counts

    ## Gene summary
    gene_summary = mu_long %>% group_by(variable, basemean, ncells, ind) %>% summarise(
      amount_zero_counts = sum(counts == 0),
      amount_cells = n(),
      mean_counts = mean(counts),
      max_counts = max(counts),
      min_counts = min(counts),
      variance_counts = var(counts)
    )

    gene_summary$percentage_zero = (gene_summary$amount_zero_counts / gene_summary$amount_cells) *100
    gene_summary =gene_summary %>% mutate(percentage_zero_bin = cut(percentage_zero, breaks=c(-0.001,5, 10, 20,40, 60, 80,90, 95, 100)))

    gene_summary_gene1 = gene_summary
    colnames(gene_summary_gene1) = paste0('gene1_', colnames(gene_summary_gene1))


    gene_summary_gene2 = gene_summary
    colnames(gene_summary_gene2) = paste0('gene2_', colnames(gene_summary_gene2))
        
    # Cell summary

    cell_summary = mu_long %>% group_by(cell_ident, ncells, ind) %>% summarise(
        amount_counts = sum(counts), amount_genes = n(), amount_zero_counts = sum(counts == 0), mean_counts = mean(counts),
        max_counts = max(counts), min_counts = min(counts), var_counts = var(counts))
    cell_summary$percentage_zero = cell_summary$amount_zero_counts/ cell_summary$amount_genes
    total_mean_counts = cell_summary %>% group_by(ncells, ind) %>% summarise(total_mean_counts = mean(mean_counts))
    cell_summary = merge(cell_summary, total_mean_counts)

    cell_summary$scaling = cell_summary$total_mean_counts/ cell_summary$mean_counts

    ### Concat gene and summary
    gene_summary_all_cells = rbind(gene_summary_all_cells, gene_summary)
    cell_summary_all_cells = rbind(cell_summary_all_cells, cell_summary)
    count_data = rbind(count_data, mu_long)
                                                                            
        
     }
    

write.csv(   gene_summary_all_cells ,gzfile( paste0(output_path_sc, 'A1_4_Gene_Summary', '_', use_approach, '_',  variant,  '.csv.gz')))
write.csv(cell_summary_all_cells, gzfile(paste0(output_path_sc, 'A1_4_Cell_Summary', '_',use_approach, '_',  variant, '.csv.gz')))    
write.csv(count_data, gzfile(paste0(output_path_sc, '/A1_4_Simulated_Counts', use_approach,  variant,  '.csv.gz')))

[1] "XX;1"


`summarise()` has grouped output by 'variable', 'basemean', 'ncells'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'cell_ident', 'ncells'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'ncells'. You can override using the
`.groups` argument.


[1] "XX;2"


`summarise()` has grouped output by 'variable', 'basemean', 'ncells'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'cell_ident', 'ncells'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'ncells'. You can override using the
`.groups` argument.


[1] "XX;7"


...

`summarise()` has grouped output by 'variable', 'basemean', 'ncells'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'cell_ident', 'ncells'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'ncells'. You can override using the
`.groups` argument.


[1] "YY;9"


`summarise()` has grouped output by 'variable', 'basemean', 'ncells'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'cell_ident', 'ncells'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'ncells'. You can override using the
`.groups` argument.


[1] "YY;10"


`summarise()` has grouped output by 'variable', 'basemean', 'ncells'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'cell_ident', 'ncells'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'ncells'. You can override using the
`.groups` argument.


In [ ]:
head(mu_long,2)

variable,cell_ident,value,scaling,mu,size,counts,basemean,ncells,ind
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
gene1_1,cell_YY;10_10_1,0.4144232,0.922164,27.16667,12.45769,26,27.16667,10,YY;10
gene1_1,cell_YY;10_10_8,0.1896982,1.289884,27.16667,12.45769,14,27.16667,10,YY;10


In [ ]:
#mu_long %>% group_by(cell_ident) %>% count()

In [ ]:
head(gene_summary,2)

variable,basemean,ncells,ind,amount_zero_counts,amount_cells,mean_counts,max_counts,min_counts,variance_counts,percentage_zero,percentage_zero_bin
<fct>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
gene1_1,27.166667,10,YY;10,0,10,23.8,49,11,161.511111,0,"(-0.001,5]"
gene2_1,1.944429,10,YY;10,3,10,1.6,3,0,1.822222,30,"(20,40]"
